In [1]:
import os
import mysql.connector
import csv
import requests
import os

from tqdm import tqdm
import cv2
import numpy as np
from shutil import copyfile
import subprocess, shlex
import random
import albumentations as A
from sklearn.model_selection import train_test_split
from itertools import groupby

In [ ]:
#download video from url

In [ ]:
 

with open("invalidpatch.csv","r",newline='') as file:
    reader = csv.reader(file,delimiter=',')
    for row in tqdm(reader):
 
        name = row[1].split("/")[-1]
        path = "/home/jy/projects/yolo/dataprepare/patch1/"+row[0]+"/"+name
        r = requests.get(row[1], allow_redirects=True)
        if not os.path.exists("/home/jy/projects/yolo/dataprepare/patch1/"+row[0]+"/"):
            os.makedirs("/home/jy/projects/yolo/dataprepare/patch1/"+row[0]+"/")
        if not os.path.exists("/home/jy/projects/yolo/dataprepare/patch1/"+row[0]+"/"+name):
            open(path, 'wb').write(r.content)

In [ ]:
#rename and unzip

In [36]:
for root,dir,files in os.walk("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_zip"):
    for f in files:
        if 'zip.zip' in f:
            os.rename("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_zip/"+f,"/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_zip/"+f.split(".")[0]+".zip")

In [37]:
#rename and unzip thridbatch
for root,dir,files in os.walk("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_zip"):
    for f in files:
        print(f)
        dirname = f.split(".")[0]
#         os.rename("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_zip/"+f,"/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_zip/"+f.split(" ")[0]+".zip")
 
#         os.mkdir('/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_unzip/')
        subprocess.run(shlex.split("mkdir "+"/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_unzip/"+dirname))
        subprocess.run(shlex.split("unzip /home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_zip/"+f+" -d /home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_unzip/"+dirname))
#         print("unzip /home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_unzip/"+f+" -d /home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_unzip/"+dirname)
        

task_d54809863_valid_2-2021_01_12_07_39_16-yolo.zip
task_d88628921_valid_5-2021_01_15_06_25_01-yolo.zip
task_c90841818_valid_2-2021_01_14_03_37_13-yolo.zip
task_e57378973_valid_5-2021_01_14_07_53_54-yolo.zip
task_d22710481_valid_1-2020_12_18_09_39_44-yolo.zip
task_d22710484_valid_2-2021_01_15_03_36_52-yolo.zip
task_d81141469_valid_5-2020_12_18_08_13_09-yolo.zip
task_c90842327_valid_2-2021_01_15_02_02_12-yolo.zip
task_e57379409_valid_3-2021_01_15_06_36_04-yolo.zip
task_d81139089_valid_4-2021_01_12_09_07_13-yolo.zip
task_d81142728_valid_3-2020_12_18_07_54_44-yolo.zip
task_c90843625_valid_9-2020_12_29_06_08_38-yolo.zip
task_d81141499_valid_1-2020_12_18_06_02_27-yolo.zip
task_c94890223_valid_4-2021_01_12_07_54_40-yolo.zip
task_e82843260_valid_4-2021_01_12_08_48_20-yolo.zip
task_d00269541_valid_4-2021_01_12_08_30_22-yolo.zip
task_e57379192_valid_5-2020_12_18_08_58_36-yolo.zip
task_d81141708_valid_1-2020_12_18_07_23_32-yolo.zip
task_d22710529_valid_2-2021_01_12_07_33_39-yolo.zip
task_c908408

In [16]:
for root,dir,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_zip"):
    for f in files:
        dirname = f.split(".")[0]
#         os.rename("/home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_zip/"+f,"/home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_zip/"+f.split(" ")[0]+".zip")
 
#         os.mkdir('/home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_unzip/')
        subprocess.run(shlex.split("mkdir "+"/home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_unzip/"+dirname))
#         print("unzip /home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_zip/"+f+" -d /home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_unzip/"+dirname)
        subprocess.run(shlex.split("unzip /home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_zip/"+f+" -d /home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_unzip/"+dirname))
        

In [8]:
#syc label to o/c/s/obj/wash

In [48]:
# thirdbatch
for root,dir,files in os.walk("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_unzip/"):
    dirs = [d for d in dir if len(d)>20]
    for d in dirs:
        path_d = os.path.join(root,d)
        objname = os.path.join(path_d,'obj.names')
        with open(objname,'r') as objnamefile:
            names = objnamefile.readlines()
        namedict = {}
        for idx, name in enumerate(names):
            name = name.rstrip()
            namedict[name]=idx
        try: 
            namedict['wash']
        except KeyError:
            namedict['wash']=4
            
        correct_order = [namedict['lidopen'],namedict['lidclose'],namedict['sink'],namedict['lidopenwithobj'],namedict['wash']]
        txtlist = [txt for txt in os.listdir(os.path.join(path_d,'obj_train_data')) if txt[-3:]=='txt']
        for txt in txtlist:
            path_txt = os.path.join(path_d,'obj_train_data',txt)
             
            with open(path_txt,'r') as txtfile:
                yolotxt = txtfile.readlines() 
            corrected = []
 
            for li in yolotxt:
                labels = str(correct_order.index(int(li[0])))
                rest = li[1:]
                corrected.append(labels+rest)
            with open(path_txt,'w') as txtfilewriter:
                for cor in corrected:
                    txtfilewriter.write(cor)

In [34]:
for root,dir,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_unzip"):
    dirs = [d for d in dir if len(d)>20]
    for d in dirs:
        path_d = os.path.join(root,d)
        objname = os.path.join(path_d,'obj.names')
        with open(objname,'r') as objnamefile:
            names = objnamefile.readlines()
        namedict = {}
        for idx, name in enumerate(names):
            name = name.rstrip()
            namedict[name]=idx
        try: 
            namedict['wash']
        except KeyError:
            namedict['wash']=4
            
        correct_order = [namedict['lidopen'],namedict['lidclose'],namedict['sink'],namedict['lidopenwithobj'],namedict['wash']]
        txtlist = [txt for txt in os.listdir(os.path.join(path_d,'obj_train_data')) if txt[-3:]=='txt']
        for txt in txtlist:
            path_txt = os.path.join(path_d,'obj_train_data',txt)
             
            with open(path_txt,'r') as txtfile:
                yolotxt = txtfile.readlines() 
            corrected = []
 
            for li in yolotxt:
                labels = str(correct_order.index(int(li[0])))
                rest = li[1:]
                corrected.append(labels+rest)
            with open(path_txt,'w') as txtfilewriter:
                for cor in corrected:
                    txtfilewriter.write(cor)

In [ ]:
#sink only data collection

In [18]:
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_unzip"):
    dirs = [d for d in dirs if len(d)>20]
    for d in dirs:
        path_images = os.path.join(root,d,'obj_train_data')
        txts = os.listdir(path_images)
        txts = [tst for tst in txts if tst[-3:]=='txt']
        sink_candi = []
        for txt in txts:
            with open(os.path.join(path_images,txt),'r') as label:
                labels = label.readlines()
            for line in labels:
                if line[0]=='0':
                    sink_candi.append(os.path.join(path_images,txt))
                    break
        try:
            samples = random.sample(sink_candi,90)
        except ValueError:
            samples = sink_candi
        
        dst_parent = "/home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_unzip_selected180"
        dst = os.path.join(dst_parent,d)
        if not os.path.exists(dst):
            os.makedirs(dst)
        for sample in samples:
            src_name = sample.split('/')[-1].split('.')[0]
            
            dst_png_name='%s_%s.PNG'%(d,src_name)
            dst_txt_name='%s_%s.txt'%(d,src_name)
            
            png_src_path = sample[:-3]+'PNG'
            txt_src_path = sample
            
            png_dst_path = os.path.join(dst,dst_png_name)
            txt_dst_path = os.path.join(dst,dst_txt_name)
            
            with open(txt_src_path,'r') as txt_src:
                txt_src_labels = txt_src.readlines()
            txt_dst_out = []
            
            with open(txt_dst_path,'w')as txt_dst_file:
                for tsl in txt_src_labels:
                    if tsl[0] == '0':
                        txt_dst_file.write(tsl)
            copyfile(png_src_path,png_dst_path)
                

In [ ]:
# move from obj_train_data to outside

In [10]:
#thirdbatch machine only  0,1,3
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_unzip/"):
    dirs = [d for d in dirs if 'obj_train' in d]

    for d in dirs:
        dirname = root.split('/')[-1]
        if not os.path.exists('/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_ocobj_selected/'+dirname):
            os.makedirs('/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_ocobj_selected/'+dirname)
        dstdir = '/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_ocobj_selected/'+dirname
        otdpath = os.path.join(root,d)
        files = os.listdir(otdpath)
        txts = [f for f in files if f[-3:]=='txt']
        o=0
        c=0
        obj=0
        o_candi = []
        c_candi = []
        obj_candi = []
        for txt in txts:
            txtpath = os.path.join(otdpath,txt)
            with open(txtpath,'r') as txtfile:
                lines = txtfile.readlines()
            for l in lines:
                if l[0]=='0':
                    o+=1
                    o_candi.append(txtpath)
                if l[0]=='1':
                    c+=1
                    c_candi.append(txtpath)
                if l[0]=='3':
                    obj+=1
                    obj_candi.append(txtpath)
        try:
            o_samples = random.sample(o_candi,80)
        except ValueError:
            o_samples = o_candi
            
        try:
            c__samples = random.sample(c_candi,80)
        except ValueError:
            c__samples = c_candi     
            
        try:
            obj__samples = random.sample(obj_candi,80)
            
        except ValueError:
            obj__samples = obj_candi
        
#         pngpath = txtpath.split('.')[0]+'.png'
        samples = [o_samples,c__samples,obj__samples]
        for sample in samples:
            for txtpath in sample:
                pngpath_src = txtpath.split('.')[0]+'.PNG'
                pngpath_dst = dstdir+'/'+dirname+'_'+txtpath.split('/')[-1].split('.')[0]+'.PNG'
                txtpath_dst = dstdir+'/'+dirname+'_'+txtpath.split('/')[-1]
                line_output = []
                with open(txtpath,'r') as label:
                    labels = label.readlines()
                for line in labels:
                    if line[0]!='2' and  line[0]!='4' :
                        line_output.append(line)
                with open(txtpath_dst, 'w') as f:
                    for item in line_output:
                        f.write("%s" % item)
#                 copyfile(txtpath,txtpath_dst)
                copyfile(pngpath_src,pngpath_dst)
#         print(o_samples)
#         print(c__samples)
#         print(obj__samples)
        
#         print("o: %s, c: %s, obj:%s" %(len(o_samples),len(c__samples),len(obj__samples)))
                
                
        
#         path = os.path.join(root,d)
#         files = os.listdir(path)
#         copyfile(png_src_path,png_dst_path)

In [41]:
# second batch label adjusting chagne 1_0 1_1 1_3 to same device
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/secondbatch_yolo_machine/"):
    for d in dirs:
        label = d.split('_')[3][0]
        dirpath = os.path.join(root,d,)
        dirfiles = os.listdir(dirpath)
        dirfiles_txt = [f for f in dirfiles if f[-3:]=='txt']
#         print(label)
        for txt in dirfiles_txt:
            txt = os.path.join(dirpath,txt)
            with open(txt,'r') as content:
                lines = content.readlines()
            
            with open(txt, 'w') as f:
                for item in lines:
                    item = item.split(' ')
                    item[0]=label
                    item = ' '.join(item)

                    f.write("%s" % item)
                
    break

In [56]:
# first batch label adjusting chagne 1_0 1_1 1_3 to same device
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/"):
    for d in dirs:
        label = d.split('_')[3][0]
        if label == 'd':
            label = d.split('_')[1][0]
        if label == 'v':
            label = d.split('_')[2][0]
        if label == 's':
            label = d.split('_')[2][0]

        if 'task_vaild' in d or 'task_notvaild' in d:
            label = d.split('_')[2][0]
        dirpath = os.path.join(root,d,)
        dirfiles = os.listdir(dirpath)
        dirfiles_txt = [f for f in dirfiles if f[-3:]=='txt']
        for txt in dirfiles_txt:
            txt = os.path.join(dirpath,txt)
            with open(txt,'r') as content:
                lines = content.readlines()
            with open(txt, 'w') as f:
                for item in lines:
                    item = item.split(' ')
                    item[0]=label
                    item = ' '.join(item)

                    f.write("%s" % item)
                
    break

In [19]:
# thrid batch label adjusting chagne 1_0 1_1 1_3 to same device
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_ocobj_selected/"):
    for d in dirs:
        label = d.split('_')[3][0]
        dirpath = os.path.join(root,d,)
        dirfiles = os.listdir(dirpath)
        dirfiles_txt = [f for f in dirfiles if f[-3:]=='txt']
        for txt in dirfiles_txt:
            txt = os.path.join(dirpath,txt)
            with open(txt,'r') as content:
                lines = content.readlines()
            with open(txt, 'w') as f:
                for item in lines:
                    item = item.split(' ')
                    item[0]=label
                    item = ' '.join(item)

                    f.write("%s" % item)
                
    break

In [ ]:
for root,dirs,files in os.walk("firstbatch/unzips/"):
    dirs = [d for d in dirs if 'obj_train' in d]
    
    for d in dirs:
        path = os.path.join(root,d)
        files = os.listdir(path)
        copyfile(png_src_path,png_dst_path)

In [ ]:
#sink only data collection fist batch

In [59]:
for root,dirs,files in os.walk("firstbatch/unzips/"):
    for d in dirs:
        if 'train'  in d :

            path_images = os.path.join(root,d)
            txts = os.listdir(path_images)
            txts = [tst for tst in txts if tst[-3:]=='txt' and 'classification' not in tst]
            sink_candi = []
            for txt in txts:
                with open(os.path.join(path_images,txt),'r') as label:
                    labels = label.readlines()
                for line in labels:
                    if line[0]=='2':
                        sink_candi.append(os.path.join(path_images,txt))
                        break
            try:
                samples = random.sample(sink_candi,180)
            except ValueError:
                samples = sink_candi

            dst_parent = "firstbatch/firstbatch_sinkonly"
            dst = os.path.join(dst_parent,d)
            if not os.path.exists(dst):
                os.makedirs(dst)
            for sample in samples:
                src_name = sample.split('/')[-1].split('.')[0]

                dst_png_name='%s_%s.PNG'%(d,src_name)
                dst_txt_name='%s_%s.txt'%(d,src_name)

                png_src_path = sample[:-3]+'PNG'
                txt_src_path = sample

                png_dst_path = os.path.join(dst,dst_png_name)
                txt_dst_path = os.path.join(dst,dst_txt_name)

                with open(txt_src_path,'r') as txt_src:
                    txt_src_labels = txt_src.readlines()
                txt_dst_out = []

                with open(txt_dst_path,'w')as txt_dst_file:
                    for tsl in txt_src_labels:
                        if tsl[0] == '2':
                            txt_dst_file.write(tsl)
                copyfile(png_src_path,png_dst_path)


In [ ]:
# sink only data augmentation

In [57]:
 transform_flip = A.Compose([
    A.HorizontalFlip(p=1),
    A.RandomBrightnessContrast(p=0.5),
], bbox_params=A.BboxParams(format='yolo'))

transform_rotate = A.Compose([
    A.RandomRotate90(p=1),
    A.RandomBrightnessContrast(p=0.5),
], bbox_params=A.BboxParams(format='yolo'))
transform_ShiftScaleRotate = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=90, p=1),
    A.RandomBrightnessContrast(p=0.5),
], bbox_params=A.BboxParams(format='yolo'))
alies_transform = ['flip','rotate','shiftrotate']

In [39]:
# for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_ocobj_selected/"):
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/secondbatch_yolo_machine"):
    
    for f in files:
        for alies in alies_transform:
            if alies in f:
                try:
                    os.remove(os.path.join(root,f))
                except:
                    pass

In [58]:
# third batch augmentation

# for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_ocobj_selected/"):
# for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/secondbatch_yolo_machine"):
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/"):
    dirs = [d for d in dirs if 'task' in d]
    
    for d in  tqdm( dirs):
        allfile = []
        dirpath = os.path.join(root,d)
        files = os.listdir(dirpath)
        files = [file for file in files if file[-3:]=='txt']
        files = [os.path.join(dirpath,file) for file in files]
#         files = [os.path.join(root,file) for file in files if file[-3:]=='txt' and 'train' not in file and 'obj' not in file]
#         allfile+=files
        allfile = files
    
        sample_num = round(len(allfile)*0.17)
        flipfiles = random.sample(allfile,sample_num)
        rotatefiles = random.sample(allfile,sample_num)
        ShiftScale = random.sample(allfile,sample_num)
        filelist = [flipfiles,rotatefiles,ShiftScale]
    
        transforms = [transform_flip,transform_rotate,transform_ShiftScaleRotate]

        for idx,file in enumerate( filelist):
            for f in file:
                txtpath =  f 
                pngpath =  f [:-3]+'PNG'
                image = cv2.imread(pngpath)

                boxes = []
        #         print(txtpath)
                try:
                    with open (txtpath,'r')as txt:
                        lines = txt.readlines()
        #                 print(txtpath)

                except FileNotFoundError:
                    print(txtpath)
                    continue
                for line in lines:
                    line = line.rstrip()
                    label = line[0]
                    box = line[2:].split(" ")
                    box = [float(b) for b in box]
                    box.append(label)
                    boxes.append(box)
                
                transform = transforms[idx]
                try:
                    transformed = transform(image=image, bboxes=boxes)
                except :
                    print(txtpath)
                    continue
                transformed_image = transformed['image']
                transformed_bboxes = transformed['bboxes']
#                 transformed_class_labels = transformed_bboxes[-1]
                
#                 print(transformed_class_labels)
#                 if '630881' in f:
#                     print(boxes)
#                     print(transformed_bboxes)

                out = []
                for b in transformed_bboxes:
                    b = list(b)
                    b.insert(0, b.pop())
                    out.append(b)
#                 print(out)
                save_prefix = str(alies_transform[idx])
                save_path = '/'.join(pngpath.split("/")[:-1])
                png_save_path = save_path+'/'+save_prefix+'_'+pngpath.split("/")[-1]
                txt_save_path = save_path+'/'+save_prefix+'_'+txtpath.split("/")[-1]
                
                
                
                with open(txt_save_path,'w') as test:
                    for o in out:
                        test.write(' '.join(map(str,o)))
                        test.write('\n')

                cv2.imwrite(png_save_path,transformed_image)

 


 94%|█████████▍| 66/70 [01:53<00:06,  1.58s/it]

/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/task_d54809682_valid_1-2020_10_27_07_38_16-yolo/task_d54809682_valid_1-2020_10_27_07_38_16-yolo_frame_003990.txt
/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/task_d54809682_valid_1-2020_10_27_07_38_16-yolo/task_d54809682_valid_1-2020_10_27_07_38_16-yolo_frame_005170.txt
/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/task_d54809682_valid_1-2020_10_27_07_38_16-yolo/task_d54809682_valid_1-2020_10_27_07_38_16-yolo_frame_005360.txt
/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/task_d54809682_valid_1-2020_10_27_07_38_16-yolo/task_d54809682_valid_1-2020_10_27_07_38_16-yolo_frame_004950.txt
/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/task_d54809682_valid_1-2020_10_27_07_38_16-yolo/task_d54809682_valid_1-2020_10_27_07_38_16-yolo_frame_004120.txt
/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/task_d54809682_valid_1-2020_10_27_07_38

 96%|█████████▌| 67/70 [01:54<00:04,  1.57s/it]

/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/task_d54809682_valid_1-2020_10_27_07_38_16-yolo/task_d54809682_valid_1-2020_10_27_07_38_16-yolo_frame_004670.txt
/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/task_d54809682_valid_1-2020_10_27_07_38_16-yolo/task_d54809682_valid_1-2020_10_27_07_38_16-yolo_frame_004350.txt
/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/task_d54809682_valid_1-2020_10_27_07_38_16-yolo/task_d54809682_valid_1-2020_10_27_07_38_16-yolo_frame_005010.txt


100%|██████████| 70/70 [02:00<00:00,  1.72s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0i

In [20]:
# second batch sink only missclassified
allfile = []
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/sink_only_secondbatch_missclassified_unzip_selected180"):
    
    files = [os.path.join(root,file) for file in files if file[-3:]=='txt' and 'train' not in file and 'obj' not in file]
    allfile+=files
#     for f in files:
# #         if len(f)<140:
#             print(f)
    
sample_num = round(len(allfile)*0.17)
flipfiles = random.sample(allfile,sample_num)
rotatefiles = random.sample(allfile,sample_num)
ShiftScale = random.sample(allfile,sample_num)
filelist = [flipfiles,rotatefiles,ShiftScale]
    
transforms = [transform_flip,transform_rotate,transform_ShiftScaleRotate]
    
for idx,file in enumerate( filelist):
    for f in tqdm( file):
        txtpath =  f 
        pngpath =  f [:-3]+'PNG'
        image = cv2.imread(pngpath)

        boxes = []
#         print(txtpath)
        try:
            with open (txtpath,'r')as txt:
                lines = txt.readlines()
#                 print(txtpath)

        except FileNotFoundError:
            print(txtpath)
            continue
        for line in lines:
            line = line.rstrip()
            box = line[2:].split(" ")
            box = [float(b) for b in box]
            box.append('2')
            boxes.append(box)
#         print(boxes)
        transform = transforms[idx]
        try:
            transformed = transform(image=image, bboxes=boxes, class_labels=['2'])
        except :
            print(txtpath)
            continue
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        
        transformed_class_labels = transformed['class_labels']
#         print(transformed_class_labels)
#         print(transformed_bboxes)
        
        out = []
        for b in transformed_bboxes:
            b = list(b)
            b.pop()
             
            b.insert(0,'2')
            out.append(b)
        
        save_prefix = str(alies_transform[idx])
        save_path = '/'.join(pngpath.split("/")[:-1])
        png_save_path = save_path+'/'+save_prefix+'_'+pngpath.split("/")[-1]
        txt_save_path = save_path+'/'+save_prefix+'_'+txtpath.split("/")[-1]
        with open(txt_save_path,'w') as test:
            for o in out:
                test.write(' '.join(map(str,o)))
                test.write('\n')
            
        cv2.imwrite(png_save_path,transformed_image)

        
#         print(save_path)
#         print(png_save_path)
#         print(txt_save_path)
#         break
#         cv2.imwrite(path+"ShiftScaleRotate_"+f,augmented_image)
        
#         print(transformed_bboxes[0][1])
#         print(transformed_image.shape)
#         h,w,_ = transformed_image.shape
#         x1 = int((float(transformed_bboxes[0][0])*w))
#         y1 = int((float(transformed_bboxes[0][1])*h))
#         xw = int((float(transformed_bboxes[0][2])*w/2))
#         xh =  int((float(transformed_bboxes[0][3]))*h/2)
#         print((x1,y1),(x1,y1),(h,w))
#         cv2.rectangle(transformed_image, (x1-xw,y1-xh), (x1+xw,y1+xh), (255,0,0),2) 

        
#         x1 = int((float(boxes[0][0])*w))
#         y1 = int((float(boxes[0][1])*h))
#         xw = int((float(boxes[0][2])*w/2))
#         xh =  int((float(boxes[0][3]))*h/2)
        
        
#         cv2.rectangle(image, (x1-xw,y1-xh), (x1+xw,y1+xh), (255,0,0),2) 
        
#         cv2.imshow('image',image)
#         cv2.imshow('transformed_image',transformed_image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()


100%|██████████| 1040/1040 [00:20<00:00, 50.09it/s]


In [ ]:
# check files

In [44]:
for root,dirs,files in os.walk("sinkonly_1and2batch"):
    files = [os.path.join(root,file) for file in files if file[-3:]=='txt']
    for f in files:
        if not os.path.exists(f [:-3]+'PNG'):
            print(f)                  

sinkonly_1and2batch/secondbatch_sinkonly/task_d88629340_invalid_0-2020_11_16_02_10_20-yolo/task_d88629340_invalid_0-2020_11_16_02_10_20-yolo_frame_002490.txt


In [21]:
# train test val split

In [72]:
allfile = []
for root,dirs,files in os.walk("sinkonly_1and2batch"):
    
    files = [os.path.join(root,file) for file in files if file[-3:]=='txt']
    allfile+=files
    
train,val = train_test_split(allfile,test_size=0.35, random_state=42)
val,test = train_test_split(val,test_size=0.33, random_state=42)

for f in train:
    dst = 'sink_only/labels/train/'
    if not os.path.isdir(dst):
        os.makedirs(dst)
    dst_file = dst+f.split('/')[-1]
    copyfile(f,dst_file)
for f in test:
    dst = 'sink_only/labels/test/'
    if not os.path.isdir(dst):
        os.makedirs(dst)
    dst_file = dst+f.split('/')[-1]
    copyfile(f,dst_file)
for f in val:
    dst = 'sink_only/labels/val/'
    if not os.path.isdir(dst):
        os.makedirs(dst)
    dst_file = dst+f.split('/')[-1]
    copyfile(f,dst_file)
    
for f in train:
    f = f [:-3]+'PNG'
    dst = 'sink_only/images/train/'
    if not os.path.isdir(dst):
        os.makedirs(dst)
    dst_file = dst+f.split('/')[-1]
    copyfile(f,dst_file)
for f in test:
    f = f [:-3]+'PNG'
    
    dst = 'sink_only/images/test/'
    if not os.path.isdir(dst):
        os.makedirs(dst)
    dst_file = dst+f.split('/')[-1]
    copyfile(f,dst_file)
for f in val:
    f = f [:-3]+'PNG'
    
    dst = 'sink_only/images/val/'
    if not os.path.isdir(dst):
        os.makedirs(dst)
    dst_file = dst+f.split('/')[-1]
    copyfile(f,dst_file)
# for file in allfile:
#     print(file)
    
    
# for rs,ds,fs in os.walk("sinkonly_1and2batch"):
    
#     for d in ds:
#         fs = os.listdir("sink_only/pots/"+d)
 
#         train,val = train_test_split(fs,test_size=0.35, random_state=42)
#         val,test = train_test_split(val,test_size=0.15, random_state=42)
        
#         if not os.path.isdir("unzips_classify/train/"+d):
#             os.mkdir("unzips_classify/train/"+d)
#         if not os.path.isdir("unzips_classify/test/"+d):
#             os.mkdir("unzips_classify/test/"+d)
#         if not os.path.isdir("unzips_classify/val/"+d):
#             os.mkdir("unzips_classify/val/"+d)
#         for i in train:
#             copyfile(os.path.join(rs,d,i),"unzips_classify/train/"+d+"/"+i)
#         for i in val:
#             copyfile(os.path.join(rs,d,i),"unzips_classify/val/"+d+"/"+i)
#         for i in test:
#             copyfile(os.path.join(rs,d,i),"unzips_classify/test/"+d+"/"+i)

In [65]:
# sink only 2 to 0

In [75]:
allfile = []
for root,dirs,files in os.walk("sinkonly_1and2batch"):
    
    files = [os.path.join(root,file) for file in files if file[-3:]=='txt']
    allfile+=files
for f in allfile:
    with open(f,'r')as readf:
        lines = readf.readlines()
    out = []
    for li in lines:
        li = '0'+li[1:]
        out.append(li)
    with open(f,'w')as writef:
        for o in out:
            writef.write(o)

In [ ]:
# second batch nohand image extraction

In [7]:
nohand_candi = []
for root,dirs,files in os.walk("secondbatch/secondbatch_unzips/"):
    dirs = [d for d in dirs if len(d)>20]
    for d in dirs:
        path_images = os.path.join(root,d,'obj_train_data')
        txts = os.listdir(path_images)
        txts = [os.path.join(path_images,tst) for tst in txts if tst[-3:]=='txt']
        for txt in txts:
            with open(txt,'r')as txtreader:
                txtlines = txtreader.readlines()
            
            for line in txtlines:
                if line[0]!='4':
                    nohand_candi.append(txt)
                    break
nohand_candi = random.sample(nohand_candi,15000)


In [9]:
for nc in nohand_candi:
    with open(nc,'r')as txtreader:
        txtlines = txtreader.readlines()
    pngpath = nc[:-3]+'PNG'
    image = cv2.imread(pngpath)
    h,w,_ = image.shape
    for idx,line in enumerate(txtlines):
        if line[0]=='2':
            sinklocation = line.split(" ")[1:]
            sinklocation = [float(i) for i in sinklocation]
            x1 = int((float(sinklocation[0])*w))
            y1 = int((float(sinklocation[1])*h))
            xw = int((float(sinklocation[2]))*w/2)
            xh = int((float(sinklocation[3]))*h/2)
            crop_img = image[y1-xh:y1+xh, x1-xw:x1+xw]
            
 
            
            
            savepath = '/home/jy/projects/imgs/dataprepare/secondbatch_classify/secondbatch_nohand'
            savepath = savepath +'/'+ pngpath.split("/")[2]
            imname = savepath+'/'+pngpath.split("/")[2]+'_'+pngpath.split("/")[-1][:-3]+'PNG'
            
            if not os.path.isdir(savepath):
                os.makedirs(savepath)
            cv2.imwrite(imname, crop_img)

In [ ]:
# hand and nohand data augmentation

In [23]:
 transform_flip = A.Compose([
    A.HorizontalFlip(p=1),
    A.RandomBrightnessContrast(p=0.5),
])

transform_rotate = A.Compose([
    A.RandomRotate90(p=1),
    A.RandomBrightnessContrast(p=0.5),
])
transform_ShiftScaleRotate = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=90, p=1),
    A.RandomBrightnessContrast(p=0.5),
])
alies_transform = ['flip','rotate','shiftrotate']

In [24]:
allfile = []
for root,dirs,files in os.walk("handclassify"):
    
    files = [os.path.join(root,file) for file in files if file[-3:]=='PNG']
    allfile+=files
#     for f in files:
# #         if len(f)<140:
#             print(f)
    
sample_num = round(len(allfile)*0.17)
flipfiles = random.sample(allfile,sample_num)
rotatefiles = random.sample(allfile,sample_num)
ShiftScale = random.sample(allfile,sample_num)
filelist = [flipfiles,rotatefiles,ShiftScale]
    
transforms = [transform_flip,transform_rotate,transform_ShiftScaleRotate]
    
for idx,file in enumerate( filelist):
    for f in tqdm( file):
         
        pngpath =  f [:-3]+'PNG'
        image = cv2.imread(pngpath)

        transform = transforms[idx]
        try:
            transformed = transform(image=image)
        except :
            print(txtpath)
            continue
        transformed_image = transformed['image']
        
#         print(transformed_class_labels)
#         print(transformed_bboxes)
        
       
        
        save_prefix = str(alies_transform[idx])
        save_path = '/'.join(pngpath.split("/")[:-1])
        png_save_path = save_path+'/'+save_prefix+'_'+pngpath.split("/")[-1]
 
        cv2.imwrite(png_save_path,transformed_image)


 31%|███▏      | 4416/14034 [00:05<00:13, 736.56it/s]


KeyboardInterrupt: 

In [ ]:
# move hand 1&2 together

In [13]:
for root,dirs,files in os.walk("handclassify1&2/secondbatch_nohand"):
    files = [os.path.join(root,file) for file in files if file[-3:]=='PNG']
    for f in files:
        
        copyfile(f,'handclassify1&2/sec_nohand/'+f.split('/')[-2]+f.split('/')[-1])

In [14]:
for root,dirs,files in os.walk("handclassify1&2"):
    for d in dirs:
        if '_nohand' in d:
            print('nohand')
            print(d)
 
            for r,ds,fs in os.walk(path):
                fs = [os.path.join(r,file) for file in fs if file[-3:]=='PNG']
                print(fs[0])
                
                for f in fs:
                    filename = f.split('/')[-1]
                    copyfile(f,'handclassify/nohand/'+filename)
        elif  '_hand' in d: 
            print('hand')
            print(d)
            
            path = os.path.join(root,d)
            for r,ds,fs in os.walk(path):
                fs = [os.path.join(r,file) for file in fs if file[-3:]=='PNG']
                for f in fs:
                    filename = f.split('/')[-1]
                    copyfile(f,'handclassify/hand/'+filename)

nohand
sec_nohand
hand
sec_hand
handclassify1&2/sec_hand/task_d81141612_invalid_2-2020_11_20_01_57_12-yoloframe_002400.PNG
hand
firstbatch_hand
handclassify1&2/firstbatch_hand/flip_task_vaild_5_0-2020_10_28_04_08_36-yolo_frame_024850.PNG
nohand
firstbatch_nohand
handclassify1&2/firstbatch_hand/flip_task_vaild_5_0-2020_10_28_04_08_36-yolo_frame_024850.PNG


In [ ]:
# hand classify train test val split

In [15]:
for root,dirs,files in os.walk("handclassify"):
    for d in dirs:
        path = os.path.join(root,d)
        images = os.listdir(path)
        images = [os.path.join(path,file) for file in images if file[-3:]=='PNG']
        train,val = train_test_split(images,test_size=0.35, random_state=42)
        val,test = train_test_split(val,test_size=0.33, random_state=42)
        for f in train:
            dst ='train/%s/'%d
            if not os.path.isdir(dst):
                os.makedirs(dst)
            dst_file = dst+f.split('/')[-1]
            copyfile(f,dst_file)
        for f in test:
            dst ='test/%s/'%d
            if not os.path.isdir(dst):
                os.makedirs(dst)
            dst_file = dst+f.split('/')[-1]
            copyfile(f,dst_file)
        for f in val:
            dst = 'val/%s/'%d
            if not os.path.isdir(dst):
                os.makedirs(dst)
            dst_file = dst+f.split('/')[-1]
            copyfile(f,dst_file)
    


In [ ]:
# second batch individual o/c/obj selection

In [23]:
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/second_batch_individual"):
    dirs = [d for d in dirs if d in ['1','2','3','4','5']]
    dirpaths = [os.path.join(root,d) for d in dirs ]
#     print(dirpaths)
    for dirpath in dirpaths:
        cpath = os.path.join(dirpath,'lidopen') 
        opath = os.path.join(dirpath,'lidclose') 
        objpath = os.path.join(dirpath,'lidopenwithobj') 
        folders = [cpath,opath,objpath]
        for folder in folders:
            picfolder = os.path.join(folder,'picture') 
            txtfolder = os.path.join(folder,'txt')
            pictures = os.listdir(picfolder)
            pictures.sort() 
            res = [list(i) for j, i in groupby(pictures, lambda a: a.split('_')[1])]  
            for group in res:
                try:
                    o_samples = random.sample(group,80)
                except ValueError:
                    o_samples = group
                for sample in o_samples:
                    pngpath = os.path.join(picfolder,sample)  
                    txtname = sample.split('.')[0]+'.txt'
                    txtpath = os.path.join(txtfolder,txtname)  
                    dirname = "_".join(sample.split('_')[:-2])
                    dst = '/home/jy/projects/imgs/dataprepare/secondbatch/secondbatch_yolo_machine/'+dirname
                    if not os.path.exists(dst):
                        os.makedirs(dst)
                    pngpath_dst = os.path.join(dst,sample)  
                    txtpath_dst = os.path.join(dst,txtname)  
                    line_output=[]
                    with open(txtpath,'r') as label:
                        labels = label.readlines()
                    for line in labels:
                        if line[0]!='2' and  line[0]!='4' :
                            line_output.append(line)
                    with open(txtpath_dst, 'w') as f:
                        for item in line_output:
                            f.write("%s" % item)
                    copyfile(pngpath,pngpath_dst)
#                     copyfile(txtpath,txtpath_dst)
         
    break 


In [ ]:
# first batch yolo train dataprepare

In [31]:
# first batch remvoe classfi and add obj train data folder
for root,dirs,files in os.walk('firstbatch/unzips'):
#     for d in dirs:
#         if 'train' in d:
#             file = os.listdir(os.path.join(root,d))
#             os.mkdir(os.path.join(root,d,'obj_train_data'))
#             for f in file:
#                 os.rename(os.path.join(root,d,f),os.path.join(root,d,'obj_train_data',f))
#     for d in dirs:
    for f in files:
        if 'classif' in f:
            os.remove(os.path.join(root,f))
#     break 

In [38]:
# frist batch yolo train selection 

for root,dirs,files in os.walk("firstbatch/unzips"):
    dirs = [d for d in dirs if 'obj_train' in d]

    for d in dirs:
        dirname = root.split('/')[-1]
        if not os.path.exists('firstbatch_selected/'+dirname):
            os.makedirs('firstbatch_selected/'+dirname)
        dstdir = 'firstbatch_selected/'+dirname
        otdpath = os.path.join(root,d)
        files = os.listdir(otdpath)
        txts = [f for f in files if f[-3:]=='txt']
        o=0
        c=0
        obj=0
        o_candi = []
        c_candi = []
        obj_candi = []
        for txt in txts:
            txtpath = os.path.join(otdpath,txt)
            with open(txtpath,'r') as txtfile:
                lines = txtfile.readlines()
            for l in lines:
                if l[0]=='0':
                    o+=1
                    o_candi.append(txtpath)
                if l[0]=='1':
                    c+=1
                    c_candi.append(txtpath)
                if l[0]=='3':
                    obj+=1
                    obj_candi.append(txtpath)
        try:
            if 'train' in dirname and 'obj' not in dirname:
                print(dirname)
                o_samples = random.sample(o_candi,240)
            else:
                o_samples = random.sample(o_candi,80)
        except ValueError:
            o_samples = o_candi
            
        try:
            if 'train' in dirname and 'obj' not in dirname:
                print(dirname)
                c__samples = random.sample(c_candi,240)
            else:
                c__samples = random.sample(c_candi,80)
        except ValueError:
            c__samples = c_candi     
            
        try:
            if 'train' in dirname and 'obj' not in dirname:
                print(dirname)
                obj__samples = random.sample(obj_candi,240)
            else:
                obj__samples = random.sample(obj_candi,80)
            
        except ValueError:
            obj__samples = obj_candi
        
#         pngpath = txtpath.split('.')[0]+'.png'
        samples = [o_samples,c__samples,obj__samples]
        for sample in samples:
            for txtpath in sample:
                pngpath_src = txtpath.split('.')[0]+'.PNG'
                pngpath_dst = dstdir+'/'+dirname+'_'+txtpath.split('/')[-1].split('.')[0]+'.PNG'
                txtpath_dst = dstdir+'/'+dirname+'_'+txtpath.split('/')[-1]
                line_output = []
                with open(txtpath,'r') as label:
                    labels = label.readlines()
                for line in labels:
                    if line[0]!='2' and  line[0]!='4' :
                        line_output.append(line)
                with open(txtpath_dst, 'w') as f:
                    for item in line_output:
                        f.write("%s" % item)
#                 copyfile(txtpath,txtpath_dst)
                copyfile(pngpath_src,pngpath_dst)

train3_5_0_d00000000
train3_5_0_d00000000
train3_5_0_d00000000
train2_1_1_d00000000
train2_1_1_d00000000
train2_1_1_d00000000
train6_1_1_d00000000
train6_1_1_d00000000
train6_1_1_d00000000
train7_5_0_d00000000
train7_5_0_d00000000
train7_5_0_d00000000
train0_2_0_d00000000
train0_2_0_d00000000
train0_2_0_d00000000
train5_2_0_d00000000
train5_2_0_d00000000
train5_2_0_d00000000
train3_1_0_d00000000
train3_1_0_d00000000
train3_1_0_d00000000
train8_2_0_d00000000
train8_2_0_d00000000
train8_2_0_d00000000
train6_2_0_d00000000
train6_2_0_d00000000
train6_2_0_d00000000
train2_1_0_d00000000
train2_1_0_d00000000
train2_1_0_d00000000
train7_1_0_d00000000
train7_1_0_d00000000
train7_1_0_d00000000
train0_1_0_d00000000
train0_1_0_d00000000
train0_1_0_d00000000
train8_1_0__d00000000
train8_1_0__d00000000
train8_1_0__d00000000
train1_1_0_d00000000
train1_1_0_d00000000
train1_1_0_d00000000
train7_1_1_d00000000
train7_1_1_d00000000
train7_1_1_d00000000
train8_1_1_d00000000
train8_1_1_d00000000
train8_1_1

In [80]:
file_list = []
for root,dirs,files in os.walk('/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_ocobj_selected'):
    files = [os.path.join(root,f) for f in files if f[-3:]=='PNG']
    file_list += files
for root,dirs,files in os.walk('secondbatch/secondbatch_yolo_machine'):
    files = [os.path.join(root,f)for f in files if f[-3:]=='PNG']
    file_list += files
for root,dirs,files in os.walk('firstbatch/firstbatch_selected'):
    files = [os.path.join(root,f)for f in files if f[-3:]=='PNG']
    file_list += files

In [81]:
len(file_list)

72737

In [94]:
# move 123 together yolo
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/firstbatch/firstbatch_selected/"):
    for d in dirs:
        label = d.split('_')[3][0]
        if label == 'd':
            label = d.split('_')[1][0]
        if label == 'v':
            label = d.split('_')[2][0]
        if label == 's':
            label = d.split('_')[2][0]
        if label in ['1','2','3','4','5']:
#             print(label)
            dirpath = os.path.join(root,d,)
            dirfiles = os.listdir(dirpath)
            dirfiles_txt = [f for f in dirfiles if f[-3:]=='PNG']
    #         print(label)
            for f in dirfiles_txt:
                f = os.path.join(dirpath,f)
                txtfile = f.split('.')[0]+'.txt'

                if not os.path.exists('fullset/'+label):
                    os.makedirs('fullset/'+label)
                pngdst = 'fullset/'+label+'/'+f.split('/')[-1]
                txtdst = 'fullset/'+label+'/'+f.split('/')[-1].split('.')[0]+'.txt'
                copyfile(f,pngdst)    
                copyfile(txtfile,txtdst)
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/secondbatch_yolo_machine"):
    for d in dirs:
        label = d.split('_')[3][0]
        if label in ['1','2','3','4','5']:
            dirpath = os.path.join(root,d,)
            dirfiles = os.listdir(dirpath)
            dirfiles_txt = [f for f in dirfiles if f[-3:]=='PNG']
    #         print(label)
            for f in dirfiles_txt:
                f = os.path.join(dirpath,f)
                txtfile = f.split('.')[0]+'.txt'

                if not os.path.exists('fullset/'+label):
                    os.makedirs('fullset/'+label)
                pngdst = 'fullset/'+label+'/'+f.split('/')[-1]
                txtdst = 'fullset/'+label+'/'+f.split('/')[-1].split('.')[0]+'.txt'
                copyfile(f,pngdst)    
                copyfile(txtfile,txtdst)
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_ocobj_selected"):
    for d in dirs:
        label = d.split('_')[3][0]
        if label in ['1','2','3','4','5']:
            dirpath = os.path.join(root,d,)
            dirfiles = os.listdir(dirpath)
            dirfiles_txt = [f for f in dirfiles if f[-3:]=='PNG']
    #         print(label)
            for f in dirfiles_txt:
                f = os.path.join(dirpath,f)
                txtfile = f.split('.')[0]+'.txt'

                if not os.path.exists('fullset/'+label):
                    os.makedirs('fullset/'+label)
                pngdst = 'fullset/'+label+'/'+f.split('/')[-1]
                txtdst = 'fullset/'+label+'/'+f.split('/')[-1].split('.')[0]+'.txt'
                copyfile(f,pngdst)    
                copyfile(txtfile,txtdst)

In [71]:
for f in file_list:
    
    label = f.split('_')[-9][0]
    if 'stitche' in f:
        label = f.split('_')[-10][0]

    if 'ipy' not in f:
        try:
            int(label)
        except Exception:
            label = f.split('_')[-8][0]
        txtfile = f.split('.')[0]+'.txt'
        
        if not os.path.exists('fullset/'+label):
            os.makedirs('fullset/'+label)
        pngdst = 'fullset/'+label+'/'+f.split('/')[-1]
        txtdst = 'fullset/'+label+'/'+f.split('/')[-1].split('.')[0]+'.txt'
        copyfile(f,pngdst)    
        copyfile(txtfile,txtdst)

In [2]:
# yolo 123 train test val split
from sklearn.model_selection import train_test_split
dirs = ['1','2','3','4','5']
for d in tqdm(dirs):
    
    allfile = []
    for root,dirs,files in os.walk('fullset/'+d+'/'):

        files = [os.path.join(root,file) for file in files if file[-3:]=='txt']
        allfile+=files

    train,val = train_test_split(allfile,test_size=0.35, random_state=42)
    val,test = train_test_split(val,test_size=0.33, random_state=42)

    for f in train:
        dst = 'pots/labels/train/'
        if not os.path.isdir(dst):
            os.makedirs(dst)
        dst_file = dst+f.split('/')[-1]
        copyfile(f,dst_file)
    for f in test:
        dst = 'test/labels/'
        if not os.path.isdir(dst):
            os.makedirs(dst)
        dst_file = dst+f.split('/')[-1]
        copyfile(f,dst_file)
    for f in val:
        dst = 'pots/labels/val/'
        if not os.path.isdir(dst):
            os.makedirs(dst)
        dst_file = dst+f.split('/')[-1]
        copyfile(f,dst_file)

    for f in train:
        f = f [:-3]+'PNG'
        dst = 'pots/images/train/'
        if not os.path.isdir(dst):
            os.makedirs(dst)
        dst_file = dst+f.split('/')[-1]
        copyfile(f,dst_file)
    for f in test:
        f = f [:-3]+'PNG'

        dst = 'test/images/'
        if not os.path.isdir(dst):
            os.makedirs(dst)
        dst_file = dst+f.split('/')[-1]
        copyfile(f,dst_file)
    for f in val:
        f = f [:-3]+'PNG'

        dst = 'pots/images/val/'
        if not os.path.isdir(dst):
            os.makedirs(dst)
        dst_file = dst+f.split('/')[-1]
        copyfile(f,dst_file)

100%|██████████| 5/5 [03:52<00:00, 46.55s/it] 


In [3]:
# yolo change label 12345 to 01234
for root,_,files in os.walk('pots'):
    file = [f for f in files if f[-3:]=='txt']
    for f in tqdm(file):
        line_output=[]
        with open(os.path.join(root,f),'r') as fs:
            lines = fs.readlines()
        for l in lines:
            con = l.split(' ')
            label = con[0]
            con[0] = str(int(con[0])-1)
            line_output.append(" ".join(con))
#         print(line_output)
        with open(os.path.join(root,f), 'w') as fdd:
            for item in line_output:
                fdd.write("%s" % item)
        if label not in ['1','2','3','4','5']:
            print(label)
            print (f)
 


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 16469/16469 [00:03<00:00, 4557.43it/s]


In [ ]:
# second batch classification data selection 

In [52]:
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/secondbatch/second_batch_individual"):
    dirs = [d for d in dirs if d in ['1','2','3','4','5']]
    dirpaths = [os.path.join(root,d) for d in dirs ]
#     print(dirpaths)
    for dirpath in dirpaths:
        cpath = os.path.join(dirpath,'lidopen') 
        opath = os.path.join(dirpath,'lidclose') 
        objpath = os.path.join(dirpath,'lidopenwithobj') 
        folders = [cpath,opath,objpath]
        for folder in folders:
            picfolder = os.path.join(folder,'picture') 
            txtfolder = os.path.join(folder,'txt')
            pictures = os.listdir(picfolder)
            pictures.sort() 
            res = [list(i) for j, i in groupby(pictures, lambda a: a.split('_')[1])]  
            for group in res:
                try:
                    o_samples = random.sample(group,10)
                except ValueError:
                    o_samples = group
                for sample in o_samples:
                    pngpath = os.path.join(picfolder,sample)  
                    txtname = sample.split('.')[0]+'.txt'
                    txtpath = os.path.join(txtfolder,txtname)  
                    with open(txtpath,'r') as label:
                        labels = label.readlines()
                    for line in labels:
                        if line[0]!='2' and  line[0]!='4' :
                            dirname = dirpath.split('/')[-1]+'_'+line[0]
                            dst = '/home/jy/projects/imgs/dataprepare/secondbatch_classify/pots/'+dirname
#                             dst = '/home/jy/projects/imgs/dataprepare/secondbatch_classify/pots/background'
                            if not os.path.exists(dst):
                                os.makedirs(dst)
                            image = cv2.imread(pngpath)
                            h,w,_ = image.shape
                             
                            location = line.split(" ")[1:]
                            location = [float(i) for i in location]
                            x1 = int((float(location[0])*w))
                            y1 = int((float(location[1])*h))
                            xw = int((float(location[2]))*w/2)
                            xh = int((float(location[3]))*h/2)
                            
#                             offset= random.uniform(0, 0.5)
#                             x1 = x1 + int(xh*(2 + offset))
#                             y1 = y1 + int(xh*(2+offset))
                            
                            crop_img = image[y1-xh:y1+xh, x1-xw:x1+xw]
#                             if len(crop_img)==0:
#                                 continue
                            pngname_dst = pngpath.split('/')[-1]
                            imname = dst+'/'+'_'.join(pngname_dst.split('_')[:-2])+ '_'+dirname+'_'+'_'.join(pngname_dst.split('_')[-2:])
#                             print(imname)
                            if not os.path.isdir(dst):
                                os.makedirs(dst)
                            cv2.imwrite(imname, crop_img)
#                     pngpath_dst = os.path.join(dst,sample)  
#                     txtpath_dst = os.path.join(dst,txtname)  
#                     line_output=[]

#                     with open(txtpath_dst, 'w') as f:
#                         for item in line_output:
#                             f.write("%s" % item)
#                     copyfile(pngpath,pngpath_dst)
# #                     copyfile(txtpath,txtpath_dst)
         
    break 

error: OpenCV(4.4.0) /tmp/pip-req-build-qacpj5ci/opencv/modules/imgcodecs/src/loadsave.cpp:738: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [ ]:
# thirdbatch machine classifiy data extract
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/thirdbatch/thirdbatch_unzip/"):
    dirs = [d for d in dirs if 'obj_train' in d]

    for d in dirs:
        label = os.path.join(root,d).split('/')[-2] .split('_')[3][0]
        if label in ['1','2','3','4','5']:

            otdpath = os.path.join(root,d)
            files = os.listdir(otdpath)
            txts = [f for f in files if f[-3:]=='txt']
            o=0
            c=0
            obj=0
            o_candi = []
            c_candi = []
            obj_candi = []
            for txt in txts:
                txtpath = os.path.join(otdpath,txt)
                with open(txtpath,'r') as txtfile:
                    lines = txtfile.readlines()
                for l in lines:
                    if l[0]=='0':
                        o+=1
                        o_candi.append(txtpath)
                    if l[0]=='1':
                        c+=1
                        c_candi.append(txtpath)
                    if l[0]=='3':
                        obj+=1
                        obj_candi.append(txtpath)
            try:
                o_samples = random.sample(o_candi,80)
            except ValueError:
                o_samples = o_candi

            try:
                c__samples = random.sample(c_candi,80)
            except ValueError:
                c__samples = c_candi     

            try:
                obj__samples = random.sample(obj_candi,80)

            except ValueError:
                obj__samples = obj_candi

    #         pngpath = txtpath.split('.')[0]+'.png'
            samples = [o_samples,c__samples,obj__samples]
            

            for sample in samples:
                for txtpath in sample:
                    
                    line_output = []
                    with open(txtpath,'r') as labeltxt:
                        labels = labeltxt.readlines()
                    for line in labels:
                        if line[0]!='2' and  line[0]!='4' :
                            dirname = label+'_'+line[0]

                            if not os.path.exists('/home/jy/projects/imgs/dataprepare/thirdbatch_classify/pots/background'):
                                os.makedirs('/home/jy/projects/imgs/dataprepare/thirdbatch_classify/pots/background')
                            dstdir = '/home/jy/projects/imgs/dataprepare/thirdbatch_classify/pots/background'
#                             if not os.path.exists('/home/jy/projects/imgs/dataprepare/thirdbatch_classify/pots/'+dirname+'_background'):
#                                 os.makedirs('/home/jy/projects/imgs/dataprepare/thirdbatch_classify/pots/'+dirname)
#                             dstdir = '/home/jy/projects/imgs/dataprepare/thirdbatch_classify/pots/'+dirname
                            
                            pngpath = txtpath[0:-3]+'PNG'
                            image = cv2.imread(pngpath)
                            h,w,_ = image.shape
                             
                                
                            location = line.split(" ")[1:]
                            location = [float(i) for i in location]
                            
                             
                            
                            
                            x1 = int((float(location[0])*w))
                            y1 = int((float(location[1])*h))
                            xw = int((float(location[2]))*w/2)
                            xh = int((float(location[3]))*h/2)
#                             offset= random.uniform(0, 0.5)
#                             x1 = x1 + int(xh*(2 + offset))
#                             y1 = y1 + int(xh*(2+offset))
                            crop_img = image[y1-xh:y1+xh, x1-xw:x1+xw]
                            
            
                            pngname_dst = pngpath.split('/')[-1]
                            imname = dstdir+'/'+os.path.join(root,d).split('/')[-2]+ '_'+dirname+'_'+'_'.join(pngname_dst.split('_')[-2:])
#                             print(imname)
#                             if not os.path.isdir(dst):
#                                 os.makedirs(dst)
                            cv2.imwrite(imname, crop_img)

    #                 copyfile(txtpath,txtpath_dst)


In [46]:
crop_img

array([], shape=(0, 116, 3), dtype=uint8)

In [36]:
# first batch machine dataset extract 
for root,dirs,files in os.walk("/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots"):
    dirs = [d for d in dirs if d.split('_')[0] in ['1','2','3','4','5']]
    dirpaths = [os.path.join(root,d) for d in dirs ]
    for dirpath in dirpaths:
        print(dirpath)
        nontrain = []
        
        for rootx,dirsx,filesx in os.walk(dirpath):
            for f in filesx:
                nontrain.append(f)
            nontrain.sort() 
            res = [list(i) for j, i in groupby(nontrain, lambda a: '_'.join(a.split('_')[-5:-1]))]  

        #     for dirpath in dirpaths:

            for group in res:
                candi = []
                for image in group:

                    if 'train' not in image:

                        candi.append(image)

                try:
                    o_samples = random.sample(candi,80)
                except ValueError:
                    o_samples = candi
                for oss in o_samples:
                    dirpath_dst = dirpath.replace('pots', 'pots_selected')
                    if not os.path.exists(dirpath_dst):
                        os.makedirs(dirpath_dst)
                    copyfile(os.path.join(dirpath,oss),os.path.join(dirpath_dst,oss))

/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/1_0
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/4_0
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/2_0
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/5_0
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/3_1
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/5_3
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/2_1
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/5_1
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/4_3
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/2_3
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/3_3
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/1_1
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/4_1
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/1_3
/home/jy/projects/imgs/dataprepare/firstbatch_classify/pots/3_0


In [25]:
# machine classification augmentation 
transform_flip = A.Compose([
    A.HorizontalFlip(p=1),
    A.RandomBrightnessContrast(p=0.5),
])

transform_rotate = A.Compose([
    A.RandomRotate90(p=1),
    A.RandomBrightnessContrast(p=0.5),
])
transform_ShiftScaleRotate = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=90, p=1),
    A.RandomBrightnessContrast(p=0.5),
])
alies_transform = ['flip','rotate']

for root,dirs,files in os.walk("pots_classicifation_123batch"):
    
    for d in dirs:
        allfile = []
        flag = d.split('_')[1]
        if flag =='0' or flag=='3':
            print((os.path.join(root,d)))
            for rootx,dirsx,filesx in os.walk(os.path.join(root,d)):
                files = [os.path.join(root,d,f) for f in filesx if f[-3:]=='PNG']
                allfile+=files
#     for f in files:
# #         if len(f)<140:
#             print(f)
    
        sample_num = round(len(allfile)*0.5)
        flipfiles = random.sample(allfile,sample_num)
        rotatefiles = random.sample(allfile,sample_num)
        filelist = [flipfiles,rotatefiles]

        transforms = [transform_flip,transform_rotate]

        for idx,file in enumerate( filelist):
            for f in tqdm( file):

                pngpath =  f [:-3]+'PNG'
                image = cv2.imread(pngpath)

                transform = transforms[idx]
                try:
                    transformed = transform(image=image)
                except :
                    continue
                transformed_image = transformed['image']

        #         print(transformed_class_labels)
        #         print(transformed_bboxes)



                save_prefix = str(alies_transform[idx])
                save_path = '/'.join(pngpath.split("/")[:-1])
                png_save_path = save_path+'/'+save_prefix+'_'+pngpath.split("/")[-1]

                cv2.imwrite(png_save_path,transformed_image)

  5%|▍         | 71/1564 [00:00<00:02, 708.20it/s]

pots_classicifation_123batch/1_0


 35%|███▌      | 171/482 [00:00<00:00, 1699.23it/s]

pots_classicifation_123batch/4_0


 11%|█         | 91/812 [00:00<00:00, 906.73it/s]

...

100%|██████████| 1214/1214 [00:01<00:00, 1043.78it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  4%|▎         | 66/1822 [00:00<00:02, 649.95it/s]

pots_classicifation_123batch/1_3


 17%|█▋        | 155/934 [00:00<00:00, 1542.61it/s]

pots_classicifation_123batch/3_0


100%|██████████| 934/934 [00:00<00:00, 1296.03it/s]


In [29]:
# train test val split classification
from sklearn.model_selection import train_test_split

dirs = os.listdir('pots_classicifation_123batch')

for d in tqdm(dirs):
    
    allfile = []
    for root,dirs,files in os.walk('pots_classicifation_123batch/'+d+'/'):

        files = [os.path.join(root,file) for file in files if file[-3:]=='PNG']
        allfile+=files

    train,val = train_test_split(allfile,test_size=0.35, random_state=42)
#     val,test = train_test_split(val,test_size=0.33, random_state=42)
 

    for f in train:
        f = f [:-3]+'PNG'
        dst = 'pots_classicifation_123batch_traintestval/train/'+d+'/'
        if not os.path.isdir(dst):
            os.makedirs(dst)
        dst_file = dst+f.split('/')[-1]
        copyfile(f,dst_file)
#     for f in test:
#         f = f [:-3]+'PNG'

#         dst = 'pots_classicifation_123batch_traintestval/test/'
#         if not os.path.isdir(dst):
#             os.makedirs(dst)
#         dst_file = dst+f.split('/')[-1]
#         copyfile(f,dst_file)
    for f in val:
        f = f [:-3]+'PNG'

        dst = 'pots_classicifation_123batch_traintestval/val/'+d+'/'
        if not os.path.isdir(dst):
            os.makedirs(dst)
        dst_file = dst+f.split('/')[-1]
        copyfile(f,dst_file)

100%|██████████| 16/16 [00:04<00:00,  3.72it/s]


In [27]:
allfile

[]

KeyboardInterrupt: 

In [31]:
for root,dirs,files in os.walk('/media/jy/SUMSUNGSSD/videosultra'):
    for d in tqdm(dirs):
        dirpath = os.path.join(root,d)
        videospath = os.listdir(dirpath)
        videospath = [os.path.join(root,d,v) for v in videospath if v[-3:]=='mp4']
        labelpath = os.path.join(dirpath,'labels')
        labelpath = os.path.join(labelpath,os.listdir(labelpath)[0])
        with open(labelpath,'r') as labeltxt:
            locations = labeltxt.readlines()
        for location in locations:
            location = location.rstrip()
            location = location.split(' ')
            label = location[0]
            x1 = float(location[1])
            y1 = float(location[2])
            x2 = float(location[3])
            y2 = float(location[4])

        for vid in videospath:
            dirname = vid.split('/')[-2]
            filename = vid.split('/')[-1]
            newdir = os.path.join('/media/jy/SUMSUNGSSD/videosultra_1/',dirname)
            if os.path.exists(newdir):
                continue
            vcap = cv2.VideoCapture(vid)
            
            frame_counter = 0
            fps = int(vcap.get(cv2.CAP_PROP_FPS))
            last_frame=[]
            mse_count = 0
            last_counter = 0
            moving_avg = []
            moving_light = []
            success = True
            while success:
                try:
                    if frame_counter % (1 * fps) == 0:
                        success, current_frame = vcap.read()
                        frame_counter_in_seconds = int(frame_counter / fps)
                        # if no current_frame captured
                        if current_frame is None:
                            continue
                        h, w, _ = current_frame.shape
                        x1r = int(x1 * w)
                        y1r = int(y1 * h)
                        x2r = int(x2 * w)
                        y2r = int(y2 * h)

                        obj_crop_image = current_frame[y1r:y2r, x1r:x2r]
                        obj_crop_image = cv2.GaussianBlur(obj_crop_image, (15, 15), 0)
                        found = False
                        if len(last_frame) > 0:
                            obj_crop_image_last_frame = last_frame[y1r:y2r, x1r:x2r]
                            obj_crop_image_last_frame = cv2.GaussianBlur(obj_crop_image_last_frame, (15, 15), 0)
                            try:
                                mse_value = mse(obj_crop_image_last_frame, obj_crop_image)
                            except Exception as e:
                                print(e)
                                break
                            moving_avg.append(mse_value)
                            if mse_value > 200 :
                                mse_count+=1
                            last_counter = frame_counter_in_seconds



    #                             print(moving_avg)
    #                             print(moving_average(moving_avg, 10))


                            b, g, r = cv2.split(obj_crop_image)
                            r_g = np.count_nonzero(abs(r - g))
                            r_b = np.count_nonzero(abs(r - b))
                            g_b = np.count_nonzero(abs(g - b))
                            diff_sum = float(r_g + r_b + g_b)

                            ### finding ratio of diff_sum with respect to size of image
                            ratio = diff_sum / obj_crop_image.size
                            # print(ratio)

                            avg = np.mean(obj_crop_image)
                            moving_light.append(avg)

                            if len(moving_light) > 5:
                                diff = (moving_light[-1] - moving_light[-2]) / moving_light[-2]
                                if diff>0.31 or diff<-0.31 or ratio <0.5:

                                    found = False 

                                    moving_avg = []

                            if len(moving_avg)>0 and moving_average(moving_avg, 10)[-1]>1000:
                                found = True
                        if found:


                            try:
                                os.makedirs(newdir)
                            except FileExistsError:
                                pass
                            copyfile(vid,os.path.join(newdir,filename))
                            break
                        last_frame = current_frame
                    else:
                        success = vcap.grab()
                    frame_counter+=1
                except Exception as e:
                    print(e)
                    break
    break

 31%|███       | 129/416 [00:57<03:42,  1.29it/s]

OpenCV(4.5.1) /tmp/pip-req-build-1syr35c1/opencv/modules/imgproc/src/smooth.dispatch.cpp:617: error: (-215:Assertion failed) !_src.empty() in function 'GaussianBlur'



100%|██████████| 416/416 [13:01<00:00,  1.88s/it]


In [13]:
moving_avg

[]

In [11]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [2]:
def mse(a, b):
    """
    calculate mse between image a and b
    :param a: image
    :param b: image
    :return: mse
    """
    err = np.sum((a.astype("float") - b.astype("float")) ** 2)
    err /= float(a.shape[0] * a.shape[0])

    return err


In [5]:
videospath

['D81141170_2021_01_20_00_39_16.mp4',
 'D81141170_2021_01_20_02_52_01.mp4',
 'D81141170_2021_01_21_02_16_26.mp4',
 'D81141170_2021_01_20_00_27_16.mp4',
 'D81141170_2021_01_20_00_57_17.mp4',
 'D81141170_2021_01_22_07_06_30.mp4',
 'D81141170_2021_01_20_00_51_16.mp4',
 'D81141170_2021_01_20_01_10_03.mp4',
 'D81141170_2021_01_24_05_07_47.mp4',
 'D81141170_2021_01_22_07_18_30.mp4',
 'D81141170_2021_01_24_05_01_51.mp4',
 'D81141170_2021_01_23_11_00_40.mp4',
 'D81141170_2021_01_24_04_19_30.mp4',
 'D81141170_2021_01_24_04_22_31.mp4',
 'D81141170_2021_01_22_06_36_29.mp4']